In [ ]:
# To automatocall tune the hyperparameter

In [4]:
import tensorflow as tf
from tensorflow import keras
import keras_tuner as kt


In [5]:
(img_train, label_train), (img_test, label_test) = keras.datasets.fashion_mnist.load_data()

In [7]:
img_train.shape

(60000, 28, 28)

In [16]:
img_train.dtype

dtype('float32')

In [15]:
# Normalize pixel values between 0 and 1
img_train = img_train.astype('float32') / 255.0
img_test = img_test.astype('float32') / 255.0

In [25]:
def model_builder(hp):
    model = keras.Sequential([keras.Input(shape=(28,28))])
    
    model.add(keras.layers.Flatten())
    
    hp_units = hp.Int("units", min_value=32, max_value=512, step=32)
    model.add(keras.layers.Dense(hp_units, activation="relu"))
    model.add(keras.layers.Dense(10))

    hp_learning_rate = hp.Choice('learning_rate', values=[1e-2, 1e-3, 1e-4])

    model.compile(optimizer=keras.optimizers.Adam(learning_rate=hp_learning_rate),
                    loss=keras.losses.SparseCategoricalCrossentropy(from_logits=True),
                    metrics=['accuracy'])

    return model
    

In [26]:
tuner = kt.Hyperband(model_builder,
                     objective='val_accuracy',
                     max_epochs=10,
                     factor=3,
                     directory='my_dir',
                     project_name='intro_to_kt')

In [27]:
tuner

In [28]:
stop_early = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=5)


In [29]:
tuner.search(img_train, label_train, epochs=50, validation_split=0.2, callbacks=[stop_early])

# Get the optimal hyperparameters
best_hps=tuner.get_best_hyperparameters(num_trials=1)[0]

print(f"""
The hyperparameter search is complete. The optimal number of units in the first densely-connected
layer is {best_hps.get('units')} and the optimal learning rate for the optimizer
is {best_hps.get('learning_rate')}.
""")

Trial 30 Complete [00h 00m 15s]
val_accuracy: 0.8890833258628845

Best val_accuracy So Far: 0.8930000066757202
Total elapsed time: 00h 02m 59s

The hyperparameter search is complete. The optimal number of units in the first densely-connected
layer is 320 and the optimal learning rate for the optimizer
is 0.001.



In [30]:
# Build the model with the optimal hyperparameters and train it on the data for 50 epochs
model = tuner.hypermodel.build(best_hps)
history = model.fit(img_train, label_train, epochs=50, validation_split=0.2)

val_acc_per_epoch = history.history['val_accuracy']
best_epoch = val_acc_per_epoch.index(max(val_acc_per_epoch)) + 1
print('Best epoch: %d' % (best_epoch,))

Epoch 1/50
1500/1500 ━━━━━━━━━━━━━━━━━━━━ 2s 1ms/step - accuracy: 0.7838 - loss: 0.6155 - val_accuracy: 0.8518 - val_loss: 0.4112
Epoch 2/50
1500/1500 ━━━━━━━━━━━━━━━━━━━━ 2s 1ms/step - accuracy: 0.8635 - loss: 0.3737 - val_accuracy: 0.8572 - val_loss: 0.3959
Epoch 3/50
1500/1500 ━━━━━━━━━━━━━━━━━━━━ 2s 1ms/step - accuracy: 0.8797 - loss: 0.3302 - val_accuracy: 0.8699 - val_loss: 0.3599
Epoch 4/50
1500/1500 ━━━━━━━━━━━━━━━━━━━━ 1s 979us/step - accuracy: 0.8900 - loss: 0.3011 - val_accuracy: 0.8827 - val_loss: 0.3217
Epoch 5/50
1500/1500 ━━━━━━━━━━━━━━━━━━━━ 1s 987us/step - accuracy: 0.8944 - loss: 0.2861 - val_accuracy: 0.8842 - val_loss: 0.3243
Epoch 6/50
1500/1500 ━━━━━━━━━━━━━━━━━━━━ 2s 1ms/step - accuracy: 0.8999 - loss: 0.2683 - val_accuracy: 0.8863 - val_loss: 0.3131
Epoch 7/50
1500/1500 ━━━━━━━━━━━━━━━━━━━━ 1s 946us/step - accuracy: 0.9026 - loss: 0.2555 - val_accuracy: 0.8853 - val_loss: 0.3165
Epoch 8/50
1500/1500 ━━━━━━━━━━━━━━━━━━━━ 1s 904us/step - accuracy: 0.9059 - loss: 0

In [31]:
hypermodel = tuner.hypermodel.build(best_hps)

# Retrain the model
hypermodel.fit(img_train, label_train, epochs=best_epoch, validation_split=0.2)

Epoch 1/27
1500/1500 ━━━━━━━━━━━━━━━━━━━━ 2s 1ms/step - accuracy: 0.7823 - loss: 0.6261 - val_accuracy: 0.8542 - val_loss: 0.4095
Epoch 2/27
1500/1500 ━━━━━━━━━━━━━━━━━━━━ 2s 1ms/step - accuracy: 0.8619 - loss: 0.3822 - val_accuracy: 0.8581 - val_loss: 0.3881
Epoch 3/27
1500/1500 ━━━━━━━━━━━━━━━━━━━━ 2s 1ms/step - accuracy: 0.8765 - loss: 0.3339 - val_accuracy: 0.8637 - val_loss: 0.3725
Epoch 4/27
1500/1500 ━━━━━━━━━━━━━━━━━━━━ 1s 981us/step - accuracy: 0.8855 - loss: 0.3107 - val_accuracy: 0.8777 - val_loss: 0.3323
Epoch 5/27
1500/1500 ━━━━━━━━━━━━━━━━━━━━ 1s 979us/step - accuracy: 0.8933 - loss: 0.2849 - val_accuracy: 0.8852 - val_loss: 0.3141
Epoch 6/27
1500/1500 ━━━━━━━━━━━━━━━━━━━━ 2s 1ms/step - accuracy: 0.8963 - loss: 0.2780 - val_accuracy: 0.8850 - val_loss: 0.3292
Epoch 7/27
1500/1500 ━━━━━━━━━━━━━━━━━━━━ 2s 1ms/step - accuracy: 0.9041 - loss: 0.2575 - val_accuracy: 0.8822 - val_loss: 0.3281
Epoch 8/27
1500/1500 ━━━━━━━━━━━━━━━━━━━━ 2s 1ms/step - accuracy: 0.9051 - loss: 0.250

In [32]:
eval_result = hypermodel.evaluate(img_test, label_test)
print("[test loss, test accuracy]:", eval_result)

313/313 ━━━━━━━━━━━━━━━━━━━━ 0s 418us/step - accuracy: 0.8844 - loss: 0.4373
[test loss, test accuracy]: [0.4404502809047699, 0.8827000260353088]
